### Helper functions

In [47]:
import random
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm
import os

# -------------------------------
# Configuration and Dimensions
# -------------------------------
furniture_dims = {
    'bed': (2.0, 1.5),       # (length, width)
    'dresser': (1.0, 0.5),
    'nightstand': (0.5, 0.5),
    'table': (1.2, 0.8),
    'desk': (1.0, 0.5)       # new furniture piece
}
pillar_dims = (0.8, 0.8)     # (width, depth) of the pillar
margin = 0.2                 # fixed margin from walls
table_offset = 2             # offset to move table closer to door
desk_offset = 0.3            # offset for desk placement
dresser_door_offset = 0.7    # offset for dresser placement

# -------------------------------
# Helper Functions for Placement
# -------------------------------
def place_furniture_fixed(room_length, room_width, furniture, wall, margin=margin):
    """
    Places furniture flush against a given wall.
    """
    length, width = furniture_dims[furniture]
    if wall == 'left':
        x = margin + width/2
        y = room_width/2
    elif wall == 'right':
        x = room_length - margin - width/2
        y = room_width/2
    elif wall == 'top':
        y = room_width - margin - width/2
        x = room_length/2
    else:  # bottom
        y = margin + width/2
        x = room_length/2
    return (round(x,2), round(y,2))

def place_nightstand_left(bed_center, bed_wall, margin=margin):
    """
    Places the nightstand on the physical left side of the bed.
    For vertical beds (if bed is flush on left/right wall), we assume the bed's long axis is vertical so
    the left side is in the negative y direction.
    For horizontal beds (if flush on top/bottom), the left side is in the negative x direction.
    """
    bed_length, bed_width = furniture_dims['bed']
    ns_length, ns_width = furniture_dims['nightstand']
    x, y = bed_center
    if bed_wall in ['left', 'right']:
        offset = (bed_length/2) + (ns_length/2) + margin
        x_new = x
        y_new = y - offset
    else:
        offset = (bed_width/2) + (ns_width/2) + margin
        x_new = x - offset
        y_new = y
    return (round(x_new,2), round(y_new,2))

def place_nightstand_adjacent(bed_center, margin=margin):
    """
    Always places the nightstand to the left of the bed.
    This ignores the bed's wall and shifts the nightstand horizontally left relative to the bed center.
    """
    bed_length, bed_width = furniture_dims['bed']
    ns_length, ns_width = furniture_dims['nightstand']
    x, y = bed_center
    # Shift left: subtract half the bed's width plus margin plus half the nightstand's width.
    x_new = x - (bed_width / 2 + ns_width / 2 + margin)
    return (round(x_new, 2), round(y, 2))

def place_table_near_door(room_length, room_width, door_wall, margin=margin, offset=table_offset):
    """
    Places the table closer to the door.
    """
    table_length, table_width = furniture_dims['table']
    if door_wall == 'left':
        x = margin + table_width/2 + offset
        y = room_width/2
    elif door_wall == 'right':
        x = room_length - margin - table_width/2 - offset
        y = room_width/2
    elif door_wall == 'top':
        y = room_width - margin - table_width/2 - offset
        x = room_length/2
    else:  # bottom
        y = margin + table_width/2 + offset
        x = room_length/2
    return (round(x,2), round(y,2))


#helper functions for stage-2
def place_dresser_right_of_door(room_length, room_width, door_wall, door_pos,
                                margin=0.2, offset=0.7):
    """
    Places the dresser on the occupant's 'right side' from the perspective
    of someone standing at the door and facing inward.

    - bottom wall => occupant faces up => occupant's right => x > door_pos
    - top wall => occupant faces down => occupant's right => x < door_pos
    - left wall => occupant faces right => occupant's right => y < door_pos
    - right wall => occupant faces left => occupant's right => y > door_pos

    The dresser is flush with the door's wall and offset in the perpendicular axis.
    Clamps the position if it goes out of bounds.
    """
    dresser_length, dresser_width = furniture_dims['dresser']

    if door_wall == 'bottom':
        # Flush with bottom wall
        # occupant faces up => occupant's right => x > door_pos
        y = margin + dresser_width / 2
        x = door_pos + (dresser_length / 2) + offset
        # clamp if out of bounds
        if x + (dresser_length / 2) > room_length - margin:
            x = room_length - margin - (dresser_length / 2)

    elif door_wall == 'top':
        # Flush with top wall
        # occupant faces down => occupant's right => x < door_pos
        y = room_width - margin - (dresser_width / 2)
        x = door_pos - (dresser_length / 2) - offset
        # clamp
        if x - (dresser_length / 2) < margin:
            x = margin + (dresser_length / 2)

    elif door_wall == 'left':
        # Flush with left wall
        # occupant faces right => occupant's right => y < door_pos
        x = margin + (dresser_width / 2)
        y = door_pos - (dresser_length / 2) - offset
        # clamp
        if y - (dresser_length / 2) < margin:
            y = margin + (dresser_length / 2)

    else:  # door_wall == 'right'
        # Flush with right wall
        # occupant faces left => occupant's right => y > door_pos
        x = room_length - margin - (dresser_width / 2)
        y = door_pos + (dresser_length / 2) + offset
        # clamp
        if y + (dresser_length / 2) > room_width - margin:
            y = room_width - margin - (dresser_length / 2)

    return (round(x, 2), round(y, 2))



def place_desk_near_window(room_length, room_width, window_wall, margin=margin, offset=desk_offset):
    """
    Places the desk flush against the unobstructed window wall.
    """
    desk_length, desk_width = furniture_dims['desk']
    if window_wall == 'top':
        x = room_length/2
        y = room_width - margin - desk_width/2 - offset
    elif window_wall == 'bottom':
        x = room_length/2
        y = margin + desk_width/2 + offset
    elif window_wall == 'left':
        x = margin + desk_width/2 + offset
        y = room_width/2
    else:  # right
        x = room_length - margin - desk_width/2 - offset
        y = room_width/2
    return (round(x,2), round(y,2))

def place_pillar_at_window(room_length, room_width, window_wall, margin=margin):
    """
    Places a pillar flush against the specified window wall.
    """
    pillar_width, pillar_depth = pillar_dims
    if window_wall == 'top':
        x = room_length/2
        y = room_width - margin - pillar_depth/2
    elif window_wall == 'bottom':
        x = room_length/2
        y = margin + pillar_depth/2
    elif window_wall == 'left':
        x = margin + pillar_width/2
        y = room_width/2
    else:  # right
        x = room_length - margin - pillar_width/2
        y = room_width/2
    return (round(x,2), round(y,2))

### Stage 1

In [ ]:
def generate_layout_sample_stage1():
    """
    Generates one Stage 1 room layout sample using deterministic rules.
    Room dimensions are restricted to [6.0, 7.0] for both length and width.
    
    Rules (Stage 1):
      - Door is always present and placed in the center of a randomly chosen wall.
      - If door is on 'left' or 'right', window is always on 'bottom';
        if door is on 'top' or 'bottom', window is always on 'right'.
      - Furniture placement:
          * If door is on 'left': bed on 'right'; dresser on 'top'; nightstand always to the left of the bed.
          * If door is on 'right': bed on 'left'; dresser on 'top'; nightstand always to the left of the bed.
          * If door is on 'top':  bed on 'bottom'; dresser on 'left'; nightstand always to the left of the bed.
          * If door is on 'bottom': bed on 'top'; dresser on 'left'; nightstand always to the left of the bed.
      - Table is placed closer to the door.
    
    For stage 1, we set features not applicable (e.g. second window, pillar, blocked windows, desk)
    to default values.
    """
    # --- Room dimensions (narrow range) ---
    room_length = round(random.uniform(6.0, 7.0), 2)
    room_width  = round(random.uniform(6.0, 7.0), 2)
    
    # --- Door ---
    door_exist = 1
    door_wall = random.choice(['left', 'right', 'top', 'bottom'])
    if door_wall in ['left', 'right']:
        door_pos = round(room_width / 2, 2)  # vertical position along left/right wall
    else:
        door_pos = round(room_length / 2, 2)   # horizontal position along top/bottom wall
    
    door_wall_left   = 1 if door_wall == 'left' else 0
    door_wall_right  = 1 if door_wall == 'right' else 0
    door_wall_top    = 1 if door_wall == 'top' else 0
    door_wall_bottom = 1 if door_wall == 'bottom' else 0
    
    # --- Window (only one window in stage 1) ---
    window_exist = 1
    if door_wall in ['left', 'right']:
        window_wall = 'bottom'
        window_pos = round(room_length / 2, 2)
    else:  # door_wall in ['top', 'bottom']
        window_wall = 'right'
        window_pos = round(room_width / 2, 2)
    window_wall_left   = 1 if window_wall == 'left' else 0
    window_wall_right  = 1 if window_wall == 'right' else 0
    window_wall_top    = 1 if window_wall == 'top' else 0
    window_wall_bottom = 1 if window_wall == 'bottom' else 0
    
    # --- Furniture Placement ---
    # Bed: placed flush opposite to the door.
    if door_wall == 'left':
        bed_wall = 'right'
    elif door_wall == 'right':
        bed_wall = 'left'
    elif door_wall == 'top':
        bed_wall = 'bottom'
    else:  # door_wall == 'bottom'
        bed_wall = 'top'
    bed_center = place_furniture_fixed(room_length, room_width, 'bed', bed_wall, margin)
    
    # Dresser: fixed wall choice based on door wall.
    if door_wall in ['left', 'right']:
        dresser_wall = 'top'
    else:
        dresser_wall = 'left'
    dresser_center = place_furniture_fixed(room_length, room_width, 'dresser', dresser_wall, margin)
    
    # Nightstand: always placed to the left of the bed.
    nightstand_center = place_nightstand_adjacent(bed_center, margin)
    
    # Table: placed closer to the door.
    table_center = place_table_near_door(room_length, room_width, door_wall, margin, table_offset)
    
    # --- Assemble Input Features ---
    # These keys match the stage 2 feature set. For Stage 1, we fill in defaults.
    features = {
        'stage': 1,
        'room_length': room_length,
        'room_width': room_width,
        'door_exist': door_exist,
        'door_wall_left': door_wall_left,
        'door_wall_right': door_wall_right,
        'door_wall_top': door_wall_top,
        'door_wall_bottom': door_wall_bottom,
        'door_pos': door_pos,
        'window_exist': window_exist,
        'window1_wall_left': window_wall_left,   # use our single window as window1
        'window1_wall_right': window_wall_right,
        'window1_wall_top': window_wall_top,
        'window1_wall_bottom': window_wall_bottom,
        'window2_wall_left': 0,                  # not applicable in stage 1
        'window2_wall_right': 0,
        'window2_wall_top': 0,
        'window2_wall_bottom': 0,
        'window1_pos': window_pos,
        'window2_pos': 0,
        'pillar_x': 0,                         # no pillar in stage 1
        'pillar_y': 0,
        'blocked_window_left': 0,
        'blocked_window_right': 0,
        'blocked_window_top': 0,
        'blocked_window_bottom': 0,
        'bed_wall': bed_wall,
        'dresser_wall': dresser_wall,
        'door_wall': door_wall,
        'window_wall': window_wall,
        'chosen_window_for_desk': window_wall  # default to our only window
    }
    
    # --- Assemble Output Furniture Coordinates ---
    outputs = {
        'bed_x': bed_center[0],
        'bed_y': bed_center[1],
        'dresser_x': dresser_center[0],
        'dresser_y': dresser_center[1],
        'nightstand_x': nightstand_center[0],
        'nightstand_y': nightstand_center[1],
        'table_x': table_center[0],
        'table_y': table_center[1],
        'desk_x': 0,   # No desk in stage 1
        'desk_y': 0
    }
    
    return {**features, **outputs}

def visualize_layout(sample, name, save_path='rule_based_visuals/'):
    """
    Visualizes the room layout:
      - Draws the room boundaries.
      - Draws the door and window as colored rectangles on the proper walls.
      - Draws furniture items (bed, dresser, nightstand, table) as labeled rectangles.
      
    This version is adapted for Stage 1, where only one window exists.
    It uses 'window1_pos' if the stage indicator is 1.
    """
    room_length = sample['room_length']
    room_width = sample['room_width']
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Draw room boundaries.
    room_rect = patches.Rectangle((0, 0), room_length, room_width, fill=False, edgecolor='black', linewidth=2)
    ax.add_patch(room_rect)
    
    thickness = 0.1  # thickness for door/window depiction
    
    # Draw door.
    if sample['door_exist'] == 1:
        door_wall = sample['door_wall']
        door_pos = sample['door_pos']
        if door_wall == 'left':
            door_rect = patches.Rectangle((-thickness, door_pos - 0.8/2), thickness, 0.8, color='brown', alpha=0.7)
        elif door_wall == 'right':
            door_rect = patches.Rectangle((room_length, door_pos - 0.8/2), thickness, 0.8, color='brown', alpha=0.7)
        elif door_wall == 'top':
            door_rect = patches.Rectangle((door_pos - 0.8/2, room_width), 0.8, thickness, color='brown', alpha=0.7)
        elif door_wall == 'bottom':
            door_rect = patches.Rectangle((door_pos - 0.8/2, -thickness), 0.8, thickness, color='brown', alpha=0.7)
        ax.add_patch(door_rect)
    
    # Draw window.
    if sample['window_exist'] == 1:
        window_wall = sample['window_wall']
        # For Stage 1, use window1_pos instead of window_pos.
        if sample.get('stage', 2) == 1:
            window_pos = sample.get('window1_pos', room_length/2)
        else:
            window_pos = sample.get('window_pos', room_length/2)
        if window_wall == 'left':
            window_rect = patches.Rectangle((-thickness, window_pos - 1.0/2), thickness, 1.0, color='blue', alpha=0.7)
        elif window_wall == 'right':
            window_rect = patches.Rectangle((room_length, window_pos - 1.0/2), thickness, 1.0, color='blue', alpha=0.7)
        elif window_wall == 'top':
            window_rect = patches.Rectangle((window_pos - 1.0/2, room_width), 1.0, thickness, color='blue', alpha=0.7)
        elif window_wall == 'bottom':
            window_rect = patches.Rectangle((window_pos - 1.0/2, -thickness), 1.0, thickness, color='blue', alpha=0.7)
        ax.add_patch(window_rect)
    
    # Helper to draw furniture rectangles.
    def draw_furniture(center, furniture, color='green'):
        length, width = furniture_dims[furniture]
        x, y = center
        bottom_left = (x - width/2, y - length/2)
        rect = patches.Rectangle(bottom_left, width, length, edgecolor=color, facecolor='none', linewidth=2)
        ax.add_patch(rect)
        ax.text(x, y, furniture, color=color, ha='center', va='center', fontsize=8)
    
    # Draw furniture.
    draw_furniture((sample['bed_x'], sample['bed_y']), 'bed', 'green')
    draw_furniture((sample['dresser_x'], sample['dresser_y']), 'dresser', 'purple')
    draw_furniture((sample['nightstand_x'], sample['nightstand_y']), 'nightstand', 'orange')
    draw_furniture((sample['table_x'], sample['table_y']), 'table', 'red')
    
    ax.set_xlim(-1, room_length + 1)
    ax.set_ylim(-1, room_width + 1)
    ax.set_aspect('equal', adjustable='box')
    ax.set_title("Rule-Based Room Layout")
    ax.set_xlabel("Length")
    ax.set_ylabel("Width")
    
    os.makedirs(save_path, exist_ok=True)
    plt.savefig(f"{save_path}{name}.png")
    plt.close()

def generate_dataset_rule_based(n_samples=1000):
    samples = [generate_layout_sample_stage1() for _ in range(n_samples)]
    return pd.DataFrame(samples)


# -------------------------------
# Main: Generate Dataset and Visualize Samples
# -------------------------------
if __name__ == '__main__':
    df = generate_dataset_rule_based(2000)
    df.to_csv('stage1_furniture_placement_dataset.csv', index=False)
    # print(df.head())
    
    for i in tqdm(range(10), desc="Visualizing samples"):
        sample = df.iloc[i].to_dict()
        visualize_layout(sample, name=f"layout_{i}", save_path='stage1_visuals/')


   stage  room_length  room_width  door_exist  door_wall_left  \
0      1         6.42        6.35           1               0   
1      1         6.58        6.67           1               0   
2      1         6.88        6.20           1               0   
3      1         6.67        6.13           1               0   
4      1         6.57        6.72           1               0   

   door_wall_right  door_wall_top  door_wall_bottom  door_pos  window_exist  \
0                1              0                 0      3.17             1   
1                0              1                 0      3.29             1   
2                0              1                 0      3.44             1   
3                0              0                 1      3.33             1   
4                0              1                 0      3.29             1   

   ...  bed_x  bed_y  dresser_x  dresser_y  nightstand_x  nightstand_y  \
0  ...   0.95   3.17       3.21       5.90         -0.25    

Visualizing samples: 100%|██████████| 10/10 [00:01<00:00,  8.27it/s]


### Stage 2

In [49]:
def generate_layout_sample_stage2():
    """
    Generates one Stage 2 layout sample:
      - Door is on a random wall.
      - Two windows are on opposite walls (not on door wall).
      - One pillar blocks one window.
      - Desk is placed in front of the unobstructed window.
      - Dresser is placed on the interior side of the door (using our new function).
      - Bed is placed flush opposite the door.
      - Nightstand is placed on the physical left side of the bed.
      - Table is placed near the door.
    """
    # --- Room Dimensions ---
    room_length = round(random.uniform(6.0, 7.0), 2)
    room_width  = round(random.uniform(6.0, 7.0), 2)
    
    # --- Door ---
    door_exist = 1
    door_wall = random.choice(['left', 'right', 'top', 'bottom'])
    if door_wall in ['left', 'right']:
        door_pos = round(room_width/2, 2)  # vertical position
    else:
        door_pos = round(room_length/2, 2)  # horizontal position
    
    door_wall_left   = 1 if door_wall=='left' else 0
    door_wall_right  = 1 if door_wall=='right' else 0
    door_wall_top    = 1 if door_wall=='top' else 0
    door_wall_bottom = 1 if door_wall=='bottom' else 0
    
    # --- Two Windows on Opposite Walls (not on door wall) ---
    if door_wall in ['left', 'right']:
        window1_wall = 'top'
        window2_wall = 'bottom'
    else:
        window1_wall = 'left'
        window2_wall = 'right'
    
    # --- Pillar: block one of the two windows (randomly chosen) ---
    blocked_window = random.choice([window1_wall, window2_wall])
    pillar_center = place_pillar_at_window(room_length, room_width, blocked_window, margin)
    
    # --- Desk: near the unobstructed window ---
    chosen_window = window2_wall if blocked_window == window1_wall else window1_wall
    desk_center = place_desk_near_window(room_length, room_width, chosen_window, margin, offset=desk_offset)
    
    # --- Dresser: placed on the interior side of the door ---
    dresser_center = place_dresser_right_of_door(room_length, room_width, door_wall, door_pos)
    
    # --- Bed: flush opposite the door ---
    if door_wall == 'left':
        bed_wall = 'right'
    elif door_wall == 'right':
        bed_wall = 'left'
    elif door_wall == 'top':
        bed_wall = 'bottom'
    else:
        bed_wall = 'top'
    bed_center = place_furniture_fixed(room_length, room_width, 'bed', bed_wall, margin)
    
    # --- Nightstand: on physical left side of bed ---
    nightstand_center = place_nightstand_left(bed_center, bed_wall, margin)
    
    # --- Table: near door ---
    table_center = place_table_near_door(room_length, room_width, door_wall, margin, table_offset)
    
    # --- Feature Encoding for Windows ---
    window1_wall_left   = 1 if window1_wall=='left' else 0
    window1_wall_right  = 1 if window1_wall=='right' else 0
    window1_wall_top    = 1 if window1_wall=='top' else 0
    window1_wall_bottom = 1 if window1_wall=='bottom' else 0
    window2_wall_left   = 1 if window2_wall=='left' else 0
    window2_wall_right  = 1 if window2_wall=='right' else 0
    window2_wall_top    = 1 if window2_wall=='top' else 0
    window2_wall_bottom = 1 if window2_wall=='bottom' else 0
    
    blocked_window_left   = 1 if blocked_window=='left' else 0
    blocked_window_right  = 1 if blocked_window=='right' else 0
    blocked_window_top    = 1 if blocked_window=='top' else 0
    blocked_window_bottom = 1 if blocked_window=='bottom' else 0
    
    # --- Assemble Features (Uniform with Stage 1) ---
    features = {
        'stage': 2,
        'room_length': room_length,
        'room_width': room_width,
        'door_exist': door_exist,
        'door_wall_left': door_wall_left,
        'door_wall_right': door_wall_right,
        'door_wall_top': door_wall_top,
        'door_wall_bottom': door_wall_bottom,
        'door_pos': door_pos,
        'window_exist': 1,  # stage 2 always has windows
        'window1_wall_left': window1_wall_left,
        'window1_wall_right': window1_wall_right,
        'window1_wall_top': window1_wall_top,
        'window1_wall_bottom': window1_wall_bottom,
        'window2_wall_left': window2_wall_left,
        'window2_wall_right': window2_wall_right,
        'window2_wall_top': window2_wall_top,
        'window2_wall_bottom': window2_wall_bottom,
        'window1_pos': room_length/2 if window1_wall in ['top','bottom'] else room_width/2,
        'window2_pos': room_length/2 if window2_wall in ['top','bottom'] else room_width/2,
        'pillar_x': pillar_center[0],
        'pillar_y': pillar_center[1],
        'blocked_window_left': blocked_window_left,
        'blocked_window_right': blocked_window_right,
        'blocked_window_top': blocked_window_top,
        'blocked_window_bottom': blocked_window_bottom,
        'bed_wall': bed_wall,
        # For uniformity, we set dresser_wall as the interior side.
        'dresser_wall': bed_wall,
        'door_wall': door_wall,
        # For window_wall, we use the chosen unobstructed window.
        'window_wall': chosen_window,
        'chosen_window_for_desk': chosen_window
    }
    
    # --- Assemble Output Furniture Coordinates ---
    outputs = {
        'bed_x': bed_center[0],
        'bed_y': bed_center[1],
        'dresser_x': dresser_center[0],
        'dresser_y': dresser_center[1],
        'nightstand_x': nightstand_center[0],
        'nightstand_y': nightstand_center[1],
        'table_x': table_center[0],
        'table_y': table_center[1],
        'desk_x': desk_center[0],
        'desk_y': desk_center[1]
    }
    
    return {**features, **outputs}


def generate_dataset_stage2(n_samples=1000):
    samples = [generate_layout_sample_stage2() for _ in range(n_samples)]
    return pd.DataFrame(samples)



# -------------------------------
# Visualization Function for Stage 2
# -------------------------------
def visualize_layout_stage2(sample, name, save_path='stage2_visuals/'):
    """
    Visualizes the Stage 2 room layout:
      - Door (brown), two windows (blue), pillar (grey),
      - Furniture: bed (green), dresser (purple), nightstand (orange), table (red), desk (magenta).
    """
    room_length = sample['room_length']
    room_width  = sample['room_width']
    fig, ax = plt.subplots(figsize=(8,8))
    
    # Draw room boundary
    room_rect = patches.Rectangle((0,0), room_length, room_width, fill=False, edgecolor='black', linewidth=2)
    ax.add_patch(room_rect)
    
    thickness = 0.1
    
    # Draw door
    if sample['door_exist'] == 1:
        door_wall = sample['door_wall']
        door_pos = sample['door_pos']
        if door_wall == 'left':
            door_rect = patches.Rectangle((-thickness, door_pos - 0.8/2), thickness, 0.8, color='brown', alpha=0.7)
        elif door_wall == 'right':
            door_rect = patches.Rectangle((room_length, door_pos - 0.8/2), thickness, 0.8, color='brown', alpha=0.7)
        elif door_wall == 'top':
            door_rect = patches.Rectangle((door_pos - 0.8/2, room_width), 0.8, thickness, color='brown', alpha=0.7)
        else:
            door_rect = patches.Rectangle((door_pos - 0.8/2, -thickness), 0.8, thickness, color='brown', alpha=0.7)
        ax.add_patch(door_rect)
    
    # Draw windows using a helper function
    def draw_window(center, wall):
        if wall in ['top','bottom']:
            w = 1.0; h = thickness
        else:
            w = thickness; h = 1.0
        bottom_left = (center[0]-w/2, center[1]-h/2)
        rect = patches.Rectangle(bottom_left, w, h, color='blue', alpha=0.7)
        ax.add_patch(rect)
    
    # Window 1
    if sample['window1_wall_top']:
        w1_center = (room_length/2, room_width - margin)
        w1_wall = 'top'
    elif sample['window1_wall_bottom']:
        w1_center = (room_length/2, margin)
        w1_wall = 'bottom'
    elif sample['window1_wall_left']:
        w1_center = (margin, room_width/2)
        w1_wall = 'left'
    else:
        w1_center = (room_length - margin, room_width/2)
        w1_wall = 'right'
    draw_window(w1_center, w1_wall)
    
    # Window 2
    if sample['window2_wall_top']:
        w2_center = (room_length/2, room_width - margin)
        w2_wall = 'top'
    elif sample['window2_wall_bottom']:
        w2_center = (room_length/2, margin)
        w2_wall = 'bottom'
    elif sample['window2_wall_left']:
        w2_center = (margin, room_width/2)
        w2_wall = 'left'
    else:
        w2_center = (room_length - margin, room_width/2)
        w2_wall = 'right'
    draw_window(w2_center, w2_wall)
    
    # Draw pillar
    pillar_center = (sample['pillar_x'], sample['pillar_y'])
    p_w, p_h = pillar_dims
    p_bl = (pillar_center[0]-p_w/2, pillar_center[1]-p_h/2)
    pillar_rect = patches.Rectangle(p_bl, p_w, p_h, color='grey', alpha=0.8)
    ax.add_patch(pillar_rect)
    
    # Furniture drawing helper
    dims = furniture_dims.copy()
    def draw_furniture(center, furniture, color):
        length, width = dims[furniture]
        cx, cy = center
        bottom_left = (cx - width/2, cy - length/2)
        rect = patches.Rectangle(bottom_left, width, length, edgecolor=color, facecolor='none', linewidth=2)
        ax.add_patch(rect)
        ax.text(cx, cy, furniture, color=color, ha='center', va='center', fontsize=8)
    
    draw_furniture((sample['bed_x'], sample['bed_y']), 'bed', 'green')
    draw_furniture((sample['dresser_x'], sample['dresser_y']), 'dresser', 'purple')
    draw_furniture((sample['nightstand_x'], sample['nightstand_y']), 'nightstand', 'orange')
    draw_furniture((sample['table_x'], sample['table_y']), 'table', 'red')
    draw_furniture((sample['desk_x'], sample['desk_y']), 'desk', 'magenta')
    
    ax.set_xlim(-1, room_length+1)
    ax.set_ylim(-1, room_width+1)
    ax.set_aspect('equal', adjustable='box')
    ax.set_title("Stage 2 Room Layout")
    ax.set_xlabel("Length")
    ax.set_ylabel("Width")
    
    os.makedirs(save_path, exist_ok=True)
    plt.savefig(f"{save_path}{name}.png")
    plt.close()

# -------------------------------
# Main: Generate Dataset and Visualize Samples
# -------------------------------
if __name__ == '__main__':
    df_stage2 = generate_dataset_stage2(2000)
    df_stage2.to_csv('stage2_furniture_placement_dataset.csv', index=False)
    print(df_stage2.head())
    
    for i in tqdm(range(10), desc="Visualizing Stage 2 samples"):
        sample = df_stage2.iloc[i].to_dict()
        visualize_layout_stage2(sample, name=f"layout_{i}", save_path='stage2_visuals/')


   stage  room_length  room_width  door_exist  door_wall_left  \
0      2         6.49        6.66           1               1   
1      2         6.55        6.95           1               0   
2      2         6.24        6.20           1               0   
3      2         6.56        6.78           1               1   
4      2         6.86        6.55           1               1   

   door_wall_right  door_wall_top  door_wall_bottom  door_pos  window_exist  \
0                0              0                 0      3.33             1   
1                1              0                 0      3.48             1   
2                0              0                 1      3.12             1   
3                0              0                 0      3.39             1   
4                0              0                 0      3.27             1   

   ...  bed_x  bed_y  dresser_x  dresser_y  nightstand_x  nightstand_y  \
0  ...   5.54   3.33       0.45       2.13          5.54    

Visualizing Stage 2 samples: 100%|██████████| 10/10 [00:01<00:00,  7.44it/s]


In [52]:
import pandas as pd

df1 = pd.read_csv('stage1_furniture_placement_dataset.csv')
df2 = pd.read_csv('stage2_furniture_placement_dataset.csv')

# Combine all CSVs
combined_df = pd.concat([df1, df2])

# Save to new CSV
combined_df.to_csv('c:/Users/sathw/Desktop/final_furniture_placement/combined_output.csv', index=False)

### Training

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load the Dataset and Preprocess
# -------------------------------
# Replace "your_dataset.csv" with the actual CSV filename.
df = pd.read_csv("combined_output.csv")

# Fill missing values (if any) with 0 (or choose another strategy)
df.fillna(0, inplace=True)

# -------------------------------
# 2. Define Input and Output Feature Names
# -------------------------------
input_features = [
    "stage",    #done
    "room_length", "room_width",    #done
    "door_exist",   #remove
    "door_wall_left", "door_wall_right", "door_wall_top", "door_wall_bottom", #done
    "door_pos", #not done
    "window_exist", #remove
    "window1_wall_left", "window1_wall_right", "window1_wall_top", "window1_wall_bottom", #done
    "window2_wall_left", "window2_wall_right", "window2_wall_top", "window2_wall_bottom", #done
    "window1_pos", "window2_pos", #not done
    "pillar_x", "pillar_y", #done
    "blocked_window_left", "blocked_window_right", "blocked_window_top", "blocked_window_bottom" #done
]

output_features = [
    "bed_x", "bed_y",
    "dresser_x", "dresser_y",
    "nightstand_x", "nightstand_y",
    "table_x", "table_y",
    "desk_x", "desk_y"
]

print("Input features used for training:", input_features)
print("Output features:", output_features)

# -------------------------------
# 3. Extract Inputs and Outputs
# -------------------------------
X = df[input_features].values   # shape: (num_samples, number_of_inputs)
y = df[output_features].values  # shape: (num_samples, 10)

# -------------------------------
# 4. Normalize the Data
# -------------------------------
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# -------------------------------
# 5. Split Data into Train, Validation, and Test Sets (80%/10%/10%)
# -------------------------------
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.1, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1111, random_state=42
)  # ~80% train, 10% val, 10% test

# -------------------------------
# 6. Convert Data to PyTorch Tensors and Create DataLoaders
# -------------------------------
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor   = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor   = torch.tensor(y_val, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset   = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset  = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# -------------------------------
# 7. Define the Model
# -------------------------------
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )
    
    def forward(self, x):
        return self.net(x)

input_dim = len(input_features)  # should be 26 (if our list above is used)
output_dim = len(output_features)  # 10
model = MLP(input_dim, output_dim)

# -------------------------------
# 8. Define Loss Function and Optimizer
# -------------------------------
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# -------------------------------
# 9. Training Loop
# -------------------------------
num_epochs = 50
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
    train_loss /= len(train_loader.dataset)
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
    val_loss /= len(val_loader.dataset)
    
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model_stage2.pth")

# -------------------------------
# 10. Test Evaluation
# -------------------------------
model.load_state_dict(torch.load("best_model_stage2.pth"))
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item() * inputs.size(0)
test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}")


Input features used for training: ['stage', 'room_length', 'room_width', 'door_exist', 'door_wall_left', 'door_wall_right', 'door_wall_top', 'door_wall_bottom', 'door_pos', 'window_exist', 'window1_wall_left', 'window1_wall_right', 'window1_wall_top', 'window1_wall_bottom', 'window2_wall_left', 'window2_wall_right', 'window2_wall_top', 'window2_wall_bottom', 'window1_pos', 'window2_pos', 'pillar_x', 'pillar_y', 'blocked_window_left', 'blocked_window_right', 'blocked_window_top', 'blocked_window_bottom']
Output features: ['bed_x', 'bed_y', 'dresser_x', 'dresser_y', 'nightstand_x', 'nightstand_y', 'table_x', 'table_y', 'desk_x', 'desk_y']
Epoch 1/50 - Train Loss: 0.6527 - Val Loss: 0.2043
Epoch 2/50 - Train Loss: 0.0680 - Val Loss: 0.0220
Epoch 3/50 - Train Loss: 0.0136 - Val Loss: 0.0071
Epoch 4/50 - Train Loss: 0.0050 - Val Loss: 0.0033
Epoch 5/50 - Train Loss: 0.0026 - Val Loss: 0.0020
Epoch 6/50 - Train Loss: 0.0016 - Val Loss: 0.0013
Epoch 7/50 - Train Loss: 0.0011 - Val Loss: 0.000

C:\Users\sathw\AppData\Local\Temp\ipykernel_3080\329821391.py:153: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model_stage2.pth"))


### Training

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib  # For saving scalers

# -------------------------------
# 1. Load the Dataset and Preprocess
# -------------------------------
# Replace "combined_output.csv" with the actual CSV filename.
df = pd.read_csv("combined_output.csv")
df.fillna(0, inplace=True)  # Fill missing values with 0

# -------------------------------
# 2. Define Input and Output Feature Names
# -------------------------------
input_features = [
    # "stage",
    "room_length", "room_width",
    # "door_exist",
    "door_wall_left", "door_wall_right", "door_wall_top", "door_wall_bottom",
    "door_pos",
    # "window_exist",
    "window1_wall_left", "window1_wall_right", "window1_wall_top", "window1_wall_bottom",
    "window2_wall_left", "window2_wall_right", "window2_wall_top", "window2_wall_bottom",
    "window1_pos", "window2_pos",
    "pillar_x", "pillar_y",
    "blocked_window_left", "blocked_window_right", "blocked_window_top", "blocked_window_bottom"
]

output_features = [
    "bed_x", "bed_y",
    "dresser_x", "dresser_y",
    "nightstand_x", "nightstand_y",
    "table_x", "table_y",
    "desk_x", "desk_y"
]

print("Input features used for training:", input_features)
print("Output features:", output_features)

# -------------------------------
# 3. Extract Inputs and Outputs
# -------------------------------
X = df[input_features].values   # shape: (num_samples, number_of_inputs)
y = df[output_features].values    # shape: (num_samples, 10)

# -------------------------------
# 4. Normalize the Data
# -------------------------------
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Save the scalers for later use
joblib.dump(scaler_X, "scaler_X.joblib")
joblib.dump(scaler_y, "scaler_y.joblib")

# -------------------------------
# 5. Split Data into Train, Validation, and Test Sets (80%/10%/10%)
# -------------------------------
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.1, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1111, random_state=42
)  # ~80% train, 10% val, 10% test

# -------------------------------
# 6. Convert Data to PyTorch Tensors and Create DataLoaders
# -------------------------------
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor   = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor   = torch.tensor(y_val, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset   = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset  = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# -------------------------------
# 7. Define the Model
# -------------------------------
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )
    
    def forward(self, x):
        return self.net(x)

input_dim = len(input_features)  # should be 26
output_dim = len(output_features)  # 10
model = MLP(input_dim, output_dim)

# -------------------------------
# 8. Define Loss Function and Optimizer
# -------------------------------
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# -------------------------------
# 9. Training Loop
# -------------------------------
num_epochs = 50
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
    train_loss /= len(train_loader.dataset)
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
    val_loss /= len(val_loader.dataset)
    
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model_stage2_with_changes.pth")

# -------------------------------
# 10. Test Evaluation
# -------------------------------
model.load_state_dict(torch.load("best_model_stage2_with_changes.pth"))
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item() * inputs.size(0)
test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}")

# -------------------------------
# 11. Save Test Data with Predictions for Visualization
# -------------------------------
# Reconstruct the original test data (inverse transform)
X_test_unscaled = scaler_X.inverse_transform(X_test)
# Generate predictions for all test samples
all_test_preds = []
with torch.no_grad():
    for inputs, _ in test_loader:
        outputs = model(inputs)
        all_test_preds.append(outputs.numpy())
all_test_preds = np.concatenate(all_test_preds, axis=0)
y_test_pred_unscaled = scaler_y.inverse_transform(all_test_preds)

# Create DataFrames
df_test_inputs = pd.DataFrame(X_test_unscaled, columns=input_features)
df_test_preds  = pd.DataFrame(y_test_pred_unscaled, columns=output_features)

# Combine them so that you have all input features plus the predicted outputs.
df_test_combined = pd.concat([df_test_inputs, df_test_preds], axis=1)
df_test_combined.to_csv("test_predictions_new.csv", index=False)
print("Saved test predictions to 'test_predictions_new.csv'")


Input features used for training: ['stage', 'room_length', 'room_width', 'door_exist', 'door_wall_left', 'door_wall_right', 'door_wall_top', 'door_wall_bottom', 'door_pos', 'window_exist', 'window1_wall_left', 'window1_wall_right', 'window1_wall_top', 'window1_wall_bottom', 'window2_wall_left', 'window2_wall_right', 'window2_wall_top', 'window2_wall_bottom', 'window1_pos', 'window2_pos', 'pillar_x', 'pillar_y', 'blocked_window_left', 'blocked_window_right', 'blocked_window_top', 'blocked_window_bottom']
Output features: ['bed_x', 'bed_y', 'dresser_x', 'dresser_y', 'nightstand_x', 'nightstand_y', 'table_x', 'table_y', 'desk_x', 'desk_y']
Epoch 1/50 - Train Loss: 0.6953 - Val Loss: 0.2679
Epoch 2/50 - Train Loss: 0.0835 - Val Loss: 0.0201
Epoch 3/50 - Train Loss: 0.0105 - Val Loss: 0.0049
Epoch 4/50 - Train Loss: 0.0038 - Val Loss: 0.0029
Epoch 5/50 - Train Loss: 0.0024 - Val Loss: 0.0019
Epoch 6/50 - Train Loss: 0.0017 - Val Loss: 0.0014
Epoch 7/50 - Train Loss: 0.0012 - Val Loss: 0.001

C:\Users\sathw\AppData\Local\Temp\ipykernel_3080\2972801590.py:156: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model_stage2_latest.

### Generate visualizations for test data

In [60]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
from tqdm import tqdm

# -------------------------------
# 1. Load the Full Original Data (with 41 columns)
# -------------------------------
df_full = pd.read_csv("combined_output.csv")
df_full.fillna(0, inplace=True)  # Ensure no missing values

# -------------------------------
# 2. Reconstruct Test Indices from the Split
# -------------------------------
# We assume that your training code used a test_size of 0.1 with random_state=42.
# We'll use the same splitting procedure on the original DataFrame.
num_samples = len(df_full)
indices = np.arange(num_samples)
_, test_indices, _, _ = train_test_split(indices, np.zeros(num_samples), test_size=0.1, random_state=42)
df_test_full = df_full.iloc[test_indices].reset_index(drop=True)

# -------------------------------
# 3. Load the Predictions from Training
# -------------------------------
# "test_predictions.csv" is the CSV file saved from your training code containing
# the predicted outputs (and the 26 numeric inputs) – a total of 36 columns.
df_preds = pd.read_csv("test_predictions.csv")
# It should contain the following output feature columns:
output_features = [
    "bed_x", "bed_y",
    "dresser_x", "dresser_y",
    "nightstand_x", "nightstand_y",
    "table_x", "table_y",
    "desk_x", "desk_y"
]

# -------------------------------
# 4. Merge the Predicted Outputs into the Full Test Data
# -------------------------------
# Here we assume that the order of rows in df_preds corresponds exactly to the order
# of rows in df_test_full. (This should be the case if the same test_split was used.)
df_test_full.update(df_preds[output_features])
# Alternatively, if you want to add/replace the output columns, you can:
for col in output_features:
    df_test_full[col] = df_preds[col]

# Save the combined full test predictions CSV.
df_test_full.to_csv("test_predictions_full.csv", index=False)
print("Saved test predictions with all columns to 'test_predictions_full.csv'.")

# -------------------------------
# 5. Visualize the Predictions
# -------------------------------
# Now, use the visualization functions.
# It is assumed that you have defined:
#    visualize_layout(sample, name, save_path)    for Stage 1,
#    visualize_layout_stage2(sample, name, save_path) for Stage 2.

# Create folders if they don't exist.
os.makedirs("visualizations_stage1", exist_ok=True)
os.makedirs("visualizations_stage2", exist_ok=True)

# Visualize a subset (for example, the first 10 samples)
num_samples_to_visualize = 10
for i, row in tqdm(df_test_full.head(num_samples_to_visualize).iterrows(), total=num_samples_to_visualize, desc="Visualizing"):
    sample = row.to_dict()
    stage = sample.get("stage", 2)  # default to stage 2 if missing
    if stage == 1:
        visualize_layout(sample, name=f"test_layout_{i}", save_path="visualizations_stage1/")
    else:
        visualize_layout_stage2(sample, name=f"test_layout_{i}", save_path="visualizations_stage2/")

print("Visualization complete. Check the 'visualizations_stage1/' and 'visualizations_stage2/' folders.")


Saved test predictions with all columns to 'test_predictions_full.csv'.


Visualizing: 100%|██████████| 10/10 [00:01<00:00,  9.47it/s]

Visualization complete. Check the 'visualizations_stage1/' and 'visualizations_stage2/' folders.
